## Step 1: Fetching episode details for TN show

In [2]:
import json, os
from tqdm import tqdm
import pandas as pd

In [1]:
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import Spotify
# Hide this if publishing code
SPOTIFY_CLIENT_ID='9353a14355e84013a72bec5f76e8c65a'
SPOTIFY_CLIENT_SECRET='bba51938dbb547f288ec5eb029c4fba8'

if not SPOTIFY_CLIENT_ID or not SPOTIFY_CLIENT_SECRET:
    raise RuntimeError("⚠️  Missing Spotify API credentials. Set SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET.")

# Create the client
auth = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)
sp = Spotify(client_credentials_manager=auth, requests_timeout=30, retries=3)
print("✅ Spotify client ready.")


✅ Spotify client ready.


In [3]:
SHOW_ID = "122imavATqSE7eCyXIcqZL"   # Trevor Noah podcast ID
MARKET = "US"                         # adjust if needed
OUT_PATH = "data/trevor_episodes.jsonl"

os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)

def fetch_all_episodes(sp, show_id, market="US"):
    """Paginate through all episodes of a Spotify show."""
    all_eps = []
    limit, offset = 50, 0
    while True:
        results = sp.show_episodes(show_id, limit=limit, offset=offset, market=market)
        eps = results.get("items", [])
        all_eps.extend(eps)
        if not results.get("next"):
            break
        offset += limit
    return all_eps

episodes = fetch_all_episodes(sp, SHOW_ID, MARKET)
print(f"✅ Retrieved {len(episodes)} episodes.")



✅ Retrieved 95 episodes.


In [4]:
records = []
with open(OUT_PATH, "w", encoding="utf-8") as f:
    for ep in episodes:
        rec = {
            "episode_id": ep["id"],
            "name": ep.get("name"),
            "description": ep.get("description"),
            "release_date": ep.get("release_date"),
            "duration_ms": ep.get("duration_ms"),
            "language": ep.get("language"),
            "url": ep.get("external_urls", {}).get("spotify"),
            "show_id": SHOW_ID
        }
        records.append(rec)
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print(f"💾 Saved metadata for {len(records)} episodes to {OUT_PATH}")

# Quick peek
df = pd.DataFrame(records)
df["description_snippet"] = df["description"].str.replace("\n"," ").str.slice(0,200)
df[["release_date","name","description_snippet","url"]].head(10)


💾 Saved metadata for 95 episodes to data/trevor_episodes.jsonl


,release_date,name,description_snippet,url
0,2025-10-21,99% Invisible and the Megachurch (featuring Gi...,Roman Mars and frequent 99% Invisible contribu...,https://open.spotify.com/episode/257sC2OwsIs9w...
1,2025-10-16,"The Real Malala: Jeans, Crushes, & Healing",Malala like you’ve never seen her -- In this d...,https://open.spotify.com/episode/1UXnBqGu6wZJP...
2,2025-10-09,Who Owns America? Bernie Sanders Says the Quie...,Senator Bernie Sanders joins Trevor to discuss...,https://open.spotify.com/episode/4DFir6obQmJI3...
3,2025-10-02,"Kara Swisher: Tech, Power, and Why You Should ...",Trevor and Eugene Khoza have a wide-ranging co...,https://open.spotify.com/episode/47uapPMiGcjnq...
4,2025-09-25,Ezra Klein: America At Its Breaking Point,New York Times journalist and political commen...,https://open.spotify.com/episode/0IhLeiYRrSFkF...
5,2025-09-18,Will AI Save Humanity or End It? with Mustafa ...,Trevor (who is also Microsoft’s “Chief Questio...,https://open.spotify.com/episode/3lHbHhoADv6cr...
6,2025-09-15,What Now Is Back!,What Now is back! Join Trevor each week for co...,https://open.spotify.com/episode/1QFuHR2EECai6...
7,2025-08-28,Between the Seasons: Stories from a South Afri...,As we gear up for our new season launch on Sep...,https://open.spotify.com/episode/5nn997mIIyi6t...
8,2025-07-10,Between the Seasons: Trump Dropping Bombs for ...,What Now is taking a summer break! Season 3 wi...,https://open.spotify.com/episode/6v5mAFeZHGESj...
9,2025-06-26,The Totally Very Real White Genocide in South ...,Television and radio host Dan Corder and comed...,https://open.spotify.com/episode/0bMCELObBRi5C...


In [5]:
import re
import pandas as pd
import spacy
from typing import List, Optional

# load spaCy model (small one is fine)
nlp = spacy.load("en_core_web_sm")

# -------------------------------
# 1. Regex-based candidate patterns
# -------------------------------
PATTERNS = [
    r"with\s+([A-Z][\w'.-]+(?:\s+[A-Z][\w'.-]+){0,3})",     # e.g. "with Barack Obama"
    r"featuring\s+([A-Z][\w'.-]+(?:\s+[A-Z][\w'.-]+){0,3})",
    r"meet\s+([A-Z][\w'.-]+(?:\s+[A-Z][\w'.-]+){0,3})",
    r"guest[:\s-]+([A-Z][\w'.-]+(?:\s+[A-Z][\w'.-]+){0,3})",
    r"joins\s+([A-Z][\w'.-]+(?:\s+[A-Z][\w'.-]+){0,3})",
    r"joined by\s+([A-Z][\w'.-]+(?:\s+[A-Z][\w'.-]+){0,3})",
]

BLACKLIST = {"trevor", "noah", "episode", "season", "bonus", "trailer"}
MAX_TOKENS = 4  # max number of words in a name


def rule_candidates(text: str) -> List[str]:
    """Return rule-based name candidates from the text."""
    cands = []
    for pat in PATTERNS:
        for m in re.finditer(pat, text, flags=re.IGNORECASE):
            cands.append(m.group(1).strip())
    return cands


def ner_candidates(text: str) -> List[str]:
    """Return PERSON entities detected by spaCy."""
    doc = nlp(text)
    return [ent.text.strip() for ent in doc.ents if ent.label_ == "PERSON"]


def plausible(name: str) -> bool:
    """Basic sanity filter for name plausibility."""
    parts = name.split()
    if not parts:
        return False
    if any(p.lower() in BLACKLIST for p in parts):
        return False
    if len(parts) > MAX_TOKENS:
        return False
    # ensure capitalization for at least the first word
    if not parts[0][0].isupper():
        return False
    return True


def extract_guest_name(title: str) -> Optional[str]:
    """Extract the most likely guest name from an episode title."""
    if not isinstance(title, str) or not title.strip():
        return None
    text = title.strip()
    # try rule-based first
    for name in rule_candidates(text):
        if plausible(name):
            return name
    # fallback: NER
    for name in ner_candidates(text):
        if plausible(name):
            return name
    return None


In [6]:
df["guest_name"] = df["name"].apply(extract_guest_name)

# Inspect results
df_result = df[["release_date", "name", "guest_name"]].copy()
display(df_result.head(20))

# How many episodes have a detected guest?
num_with_guests = df_result["guest_name"].notna().sum()
print(f"✅ Found {num_with_guests} episodes with apparent guests out of {len(df_result)} total.")


,release_date,name,guest_name
0,2025-10-21,99% Invisible and the Megachurch (featuring Gi...,Gillian Jacobs
1,2025-10-16,"The Real Malala: Jeans, Crushes, & Healing",None
2,2025-10-09,Who Owns America? Bernie Sanders Says the Quie...,Bernie Sanders
3,2025-10-02,"Kara Swisher: Tech, Power, and Why You Should ...",Kara Swisher
4,2025-09-25,Ezra Klein: America At Its Breaking Point,Ezra Klein
5,2025-09-18,Will AI Save Humanity or End It? with Mustafa ...,Mustafa Suleyman
6,2025-09-15,What Now Is Back!,None
7,2025-08-28,Between the Seasons: Stories from a South Afri...,None
8,2025-07-10,Between the Seasons: Trump Dropping Bombs for ...,None
9,2025-06-26,The Totally Very Real White Genocide in South ...,Dan Corder and Eugene


✅ Found 58 episodes with apparent guests out of 95 total.


In [7]:
# manually correct guest names that were missed
manual_updates = {
    1:  "Malala Yousafzai",
    12: "Tressie McMillan Cottom",
    21: "Scott Galloway",
    27: "Halle Berry",
    29: "Ben Winston",
    33: "Marques Brownlee",
    46: "Yuval Noah Harari",
    53: "Adam Grant",
    57: "Jody Avirgan",
    58: "Questlove",
    61: "Julia Louis-Dreyfus",
    63: "Jerrod Charmichael",
    64: "Brad Smith",
    70: "Orlando Bloom",
    90: "DaBaby"
}

for idx, name in manual_updates.items():
    if idx < len(df):
        df.at[idx, "guest_name"] = name
    else:
        print(f"⚠️ Skipped index {idx} — out of range (DataFrame has {len(df)} rows).")

print("✅ Manual guest names inserted.")


✅ Manual guest names inserted.


In [8]:
# Export df name, guest_name and description snippet to csv
df_export = df[["name", "guest_name", "description_snippet"]]
df_export.to_csv("data/trevor_guest_names.csv", index=False)

## Searching guest names in Search Endpoint

In [6]:
# Export test file
# import csv trevor_guest_names.csv
df = pd.read_csv("data/trevor_guest_names_fixed.csv")


#df_test = df[:3]       
#df_test.to_csv("data/trevor_guest_names_test.csv", index=False)

df_subset =df[df['changed']==1]

df_subset.to_csv("data/trevor_guest_names_updates.csv", index=False)

# Starting new file to not break whole flow if I have to rerun pipeline (24-10-2025) - see v2 file for flow with just updates

In [ ]:
# SEARCH SPOTIFY FOR EPISODES MATCHING GUEST NAMES
# ------------------------------------------------
# Requires:
#  - 'sp' in scope: a spotipy.Spotify client created via client credentials
#  - input CSV: /mnt/data/trevor_guest_names.csv with column 'guest_name'
# Outputs:
#  - data/guest_episode_matches.jsonl  (one JSON per match)
#  - data/guest_episode_summary.csv   (summary per guest)

import json, os, time, math
import pandas as pd
from tqdm import tqdm
from spotipy.exceptions import SpotifyException

INPUT = "data/trevor_guest_names.csv"
OUT_JSONL = "data/guest_episode_matches.jsonl"
OUT_SUMMARY = "data/guest_episode_summary.csv"

# CONFIG
LIMIT_PER_REQUEST = 50        # max allowed by Spotify per docs (range 0-50)
MAX_OFFSET = 1000             # Spotify docs limit on offset (range 0-1000)
MAX_PER_GUEST = None          # set to None to fetch all pages up to offset cap; or set e.g. 500
SLEEP_BETWEEN_REQUESTS = 0.3  # polite pause

# helper: safe search with backoff for 429
def spotify_search_episodes(sp, query, market="US", limit=50, offset=0, max_retries=5):
    backoff = 1.0
    for attempt in range(max_retries):
        try:
            res = sp.search(q=query, type="episode", market=market, limit=limit, offset=offset)
            return res
        except SpotifyException as e:
            status = getattr(e, "http_status", None)
            # spotipy raises SpotifyException; check message for 429
            if status == 429 or ("429" in str(e)):
                # extract Retry-After header if available
                retry_after = None
                try:
                    retry_after = int(e.headers.get("Retry-After"))
                except Exception:
                    retry_after = None
                wait = retry_after if retry_after is not None else backoff
                print(f"Rate limited (429). Waiting {wait} seconds before retrying (attempt {attempt+1}/{max_retries})...")
                time.sleep(wait)
                backoff *= 2
                continue
            else:
                # re-raise for other problems
                raise
    raise RuntimeError("Max retries exceeded for spotify_search_episodes")

# load guests
df_guests = pd.read_csv(INPUT)
if "guest_name" not in df_guests.columns:
    raise RuntimeError("CSV must contain 'guest_name' column")

unique_guests = df_guests["guest_name"].dropna().astype(str).map(str.strip).unique().tolist()
print(f"Searching Spotify for {len(unique_guests)} unique guest names...")

os.makedirs(os.path.dirname(OUT_JSONL), exist_ok=True)

# We'll write matches incrementally
written = 0
summary_rows = []

for guest in tqdm(unique_guests, desc="guests"):
    guest_matches = []
    # Build queries - try exact phrase first (quotes) then fallback
    queries = [f'"{guest}"']  
    # keep a set of seen episode ids to avoid duplicates across queries
    seen_episode_ids = set()
    # Determine how many items we'll attempt to fetch: if MAX_PER_GUEST set, cap pages
    items_to_fetch = MAX_PER_GUEST if MAX_PER_GUEST is not None else float("inf")
    fetched_count = 0

    for q in queries:
        offset = 0
        while True:
            # Spotify's offset must not exceed 1000 per docs
            if offset > MAX_OFFSET:
                print(f"Reached Spotify offset cap for guest {guest}. Stopping pagination at offset {offset}.")
                break

            # determine how many to request in this page
            limit = LIMIT_PER_REQUEST
            # if user set MAX_PER_GUEST, do not exceed that
            if items_to_fetch != float("inf"):
                remaining = items_to_fetch - fetched_count
                if remaining <= 0:
                    break
                limit = min(limit, max(1, int(remaining)))

            # call Spotify
            res = spotify_search_episodes(sp, q, market="US", limit=limit, offset=offset)
            eps_block = res.get("episodes", {}).get("items", [])
            total_available = res.get("episodes", {}).get("total", None)

            # process items
            for ep in eps_block:
                eid = ep.get("id")
                if not eid or eid in seen_episode_ids:
                    continue
                seen_episode_ids.add(eid)
                rec = {
                    "guest": guest,
                    "query": q,
                    "episode_id": ep.get("id"),
                    "episode_name": ep.get("name"),
                    "episode_description": ep.get("description"),
                    "episode_url": ep.get("external_urls", {}).get("spotify"),
                    "release_date": ep.get("release_date"),
                    "show_id": (ep.get("show") or {}).get("id"),
                    "show_name": (ep.get("show") or {}).get("name"),
                    "show_publisher": (ep.get("show") or {}).get("publisher"),
                    "fetched_at": time.time()
                }
                # append to file
                with open(OUT_JSONL, "a", encoding="utf-8") as fout:
                    fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
                guest_matches.append(rec)
                written += 1
                fetched_count += 1

            # progress & paging decision
            # If no 'next' (or fewer items than limit), stop paging this query
            next_url = res.get("episodes", {}).get("next")
            if not next_url:
                break

            # Next offset
            offset += limit

            # Keep polite
            time.sleep(SLEEP_BETWEEN_REQUESTS)

            # If we've hit configured MAX_PER_GUEST, stop
            if items_to_fetch != float("inf") and fetched_count >= items_to_fetch:
                break

        # if we've found some matches with exact-phrase, you might skip loose query to avoid duplicates;
        # decide policy here — currently we will still run the fallback (it dedupes by episode id).
    summary_rows.append({
        "guest": guest,
        "matches_found": len(guest_matches),
        "unique_episode_ids": len(seen_episode_ids),
        "sample_episode_id": (next(iter(seen_episode_ids)) if len(seen_episode_ids)>0 else None),
        "total_available_reported": total_available
    })

print(f"Done. Wrote {written} matching records to {OUT_JSONL}")

# write summary CSV
pd.DataFrame(summary_rows).to_csv(OUT_SUMMARY, index=False)
print(f"Wrote summary per-guest to {OUT_SUMMARY}")


Searching Spotify for 65 unique guest names...


guests: 100%|██████████| 65/65 [12:10<00:00, 11.24s/it]

Done. Wrote 32424 matching records to data/guest_episode_matches.jsonl
Wrote summary per-guest to data/guest_episode_summary.csv


In [ ]:
# Turn guest_episode_matches.jsonl into a DataFrame for analysis
records = []
with open("data/guest_episode_matches.jsonl", "r", encoding="utf-8  ") as f:
    for line in f:
        rec = json.loads(line)
        records.append(rec)
df_matches = pd.DataFrame(records)
print(f"Loaded {len(df_matches)} matching records.")
df_matches.head(10)

Loaded 66428 matching records.


,guest,query,episode_id,episode_name,episode_description,episode_url,release_date,show_id,show_name,show_publisher,fetched_at
0,Gillian Jacobs,"""Gillian Jacobs""",1IXqOEvAnjpvKdwE5zWoYl,Mr. Yuk,Mr. Yuk is a neon green circular sticker with ...,https://open.spotify.com/episode/1IXqOEvAnjpvK...,2024-04-30,None,None,None,1.761057e+09
1,Gillian Jacobs,"""Gillian Jacobs""",7FOKIo7hxiwxMBq6W4blwu,"Don’t Call Me Daddy (Kerri Kenney-Silver, Dan ...","This week, Scott is joined by bucket list gues...",https://open.spotify.com/episode/7FOKIo7hxiwxM...,2025-06-30,None,None,None,1.761057e+09
2,Gillian Jacobs,"""Gillian Jacobs""",3k5FkGBErXRX68xp0f8Nro,"Bonus Bang: Gillian Jacobs, Paul F. Tompkins, ...","This is episode 3 in our ""More-imony Tony"" ser...",https://open.spotify.com/episode/3k5FkGBErXRX6...,2025-01-23,None,None,None,1.761057e+09
3,Gillian Jacobs,"""Gillian Jacobs""",3uhR3UNbKWmN7Uhd17q00x,Taco Bell 4 with Gillian Jacobs,"Actress Gillian Jacobs (Love, Life of the Part...",https://open.spotify.com/episode/3uhR3UNbKWmN7...,2018-05-31,None,None,None,1.761057e+09
4,Gillian Jacobs,"""Gillian Jacobs""",0E7S5KF0zmDrVdwSpVxBYe,"Bonus Bang: Paul Rust, Gillian Jacobs, Paul F....","This is episode 5 of our ""Old No-Nos"" series, ...",https://open.spotify.com/episode/0E7S5KF0zmDrV...,2024-12-05,None,None,None,1.761057e+09
5,Gillian Jacobs,"""Gillian Jacobs""",4LFxdcRqtI2kuL26tvDS58,"Bonus Bang: Paul Rust, Gillian Jacobs, Paul F....","This is episode 4 of our ""Old No-No’s” series,...",https://open.spotify.com/episode/4LFxdcRqtI2ku...,2024-11-28,None,None,None,1.761057e+09
6,Gillian Jacobs,"""Gillian Jacobs""",5cAr9VuLaZ9MdqymVTQPHH,Gillian Jacobs loves dinosaurs,My guest this week is actor Gillian Jacobs.Gil...,https://open.spotify.com/episode/5cAr9VuLaZ9Md...,2023-09-18,None,None,None,1.761057e+09
7,Gillian Jacobs,"""Gillian Jacobs""",3rNGCXnfY3pNR7uxCMnmYT,Gillian Jacobs: Bad Landlord Stories (The Andy...,"Actress Gillian Jacobs (Community, The Bear) j...",https://open.spotify.com/episode/3rNGCXnfY3pNR...,2025-05-02,None,None,None,1.761057e+09
8,Gillian Jacobs,"""Gillian Jacobs""",30XXMt9SipT4qgmXsu7gZt,99% Invisible and the Megachurch (featuring Gi...,Roman Mars and frequent 99% Invisible contribu...,https://open.spotify.com/episode/30XXMt9SipT4q...,2025-10-21,None,None,None,1.761057e+09
9,Gillian Jacobs,"""Gillian Jacobs""",0VJp3uMwOeGaalB8TC6jYS,Gillian Jacobs talks Mimi-Rose Howard | HBO's ...,"This week on Girls Rewatch Podcast, we’re hitt...",https://open.spotify.com/episode/0VJp3uMwOeGaa...,2025-04-01,None,None,None,1.761057e+09


In [4]:
# Limit to only where guest name appears in episode_name

df_matches_filtered = df_matches[df_matches.apply(lambda row: row['guest'].lower() in str(row['episode_name']).lower(), axis=1)]
print(len(df_matches_filtered))

32840


## Retrieving Podcast ID from Get Episode endpoint and episode ID

In [ ]:
import os, math, time, random
import pandas as pd
from tqdm import tqdm
from spotipy.exceptions import SpotifyException

# ---- Config ----
MARKET = "US"
BATCH_SIZE = 50                      # Spotify max for episodes endpoint
CHECKPOINT_DIR = "data"
CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, "df_matches_showinfo_checkpoint.csv")
CACHE_PATH = os.path.join(CHECKPOINT_DIR, "episode_show_cache.csv")
SAVE_EVERY = 1                       # save after every batch; increase if you want fewer writes
MAX_RETRIES = 6
BACKOFF_START = 1.0                  # seconds, exponential backoff base

os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Ensure columns exist on df_matches_filtered
for col in ["show_id", "show_name", "show_publisher"]:
    if col not in df_matches_filtered.columns:
        df_matches_filtered[col] = None

# 1) Determine which episode_ids actually need fetching (de-dupe!)
need_mask = df_matches_filtered["show_id"].isna() | df_matches_filtered["show_name"].isna()
to_fetch_df = df_matches_filtered.loc[need_mask, ["episode_id"]].dropna().drop_duplicates()
episode_ids_needed = to_fetch_df["episode_id"].astype(str).tolist()
print(f"Unique episodes needing show info: {len(episode_ids_needed)}")

# 2) Load persistent cache (episode_id -> show fields)
if os.path.exists(CACHE_PATH):
    cache_df = pd.read_csv(CACHE_PATH, dtype=str)
    cache_df = cache_df.dropna(subset=["episode_id"]).drop_duplicates("episode_id")
    cache = {row["episode_id"]: {"show_id": row.get("show_id"),
                                 "show_name": row.get("show_name"),
                                 "show_publisher": row.get("show_publisher")} 
             for _, row in cache_df.iterrows()}
else:
    cache_df = pd.DataFrame(columns=["episode_id","show_id","show_name","show_publisher"])
    cache = {}

# 3) Drop ids already in cache
episode_ids_needed = [eid for eid in episode_ids_needed if eid not in cache]
print(f"After cache, still need: {len(episode_ids_needed)}")

def save_checkpoint():
    # merge cache into df and write both df + cache to disk
    if cache:
        new_rows = pd.DataFrame(
            [{"episode_id": eid,
              "show_id": v.get("show_id"),
              "show_name": v.get("show_name"),
              "show_publisher": v.get("show_publisher")} 
             for eid, v in cache.items()]
        )
        # de-dupe on write
        merged_cache = pd.concat([cache_df, new_rows], ignore_index=True)
        merged_cache = merged_cache.dropna(subset=["episode_id"]).drop_duplicates("episode_id", keep="last")
        merged_cache.to_csv(CACHE_PATH, index=False)

    # map cache into df_matches_filtered only for rows that still need it
    if cache:
        map_show_id = {k: v.get("show_id") for k, v in cache.items()}
        map_show_name = {k: v.get("show_name") for k, v in cache.items()}
        map_show_publisher = {k: v.get("show_publisher") for k, v in cache.items()}
        need_rows = df_matches_filtered["episode_id"].astype(str).isin(cache.keys())
        df_matches_filtered.loc[need_rows, "show_id"] = df_matches_filtered.loc[need_rows, "episode_id"].astype(str).map(map_show_id)
        df_matches_filtered.loc[need_rows, "show_name"] = df_matches_filtered.loc[need_rows, "episode_id"].astype(str).map(map_show_name)
        df_matches_filtered.loc[need_rows, "show_publisher"] = df_matches_filtered.loc[need_rows, "episode_id"].astype(str).map(map_show_publisher)

    df_matches_filtered.to_csv(CHECKPOINT_PATH, index=False)
    print(f"💾 Checkpoint saved → {CHECKPOINT_PATH} | Cache → {CACHE_PATH}")

def fetch_batch_with_backoff(sp, ids_batch, market=MARKET, max_retries=MAX_RETRIES):
    """Call sp.episodes(ids=...) with retry on 429 using Retry-After when available."""
    backoff = BACKOFF_START
    for attempt in range(1, max_retries + 1):
        try:
            res = sp.episodes(ids_batch, market=market)  # returns dict with "episodes": [ ... or None ... ]
            return res.get("episodes", [])
        except SpotifyException as e:
            status = getattr(e, "http_status", None)
            if status == 429 or "429" in str(e):
                # Respect Retry-After if provided
                retry_after = None
                try:
                    retry_after = int(getattr(e, "headers", {}).get("Retry-After"))  # spotipy sets headers on exception
                except Exception:
                    retry_after = None
                wait = retry_after if retry_after is not None else backoff
                wait += random.uniform(0, 0.25 * wait)  # jitter
                print(f"429 rate limit. Waiting {wait:.2f}s (attempt {attempt}/{max_retries}) ...")
                time.sleep(wait)
                backoff = min(backoff * 2, 60)  # cap backoff
                continue
            # other transient statuses can also be retried
            if status in (500, 502, 503, 504):
                wait = backoff + random.uniform(0, 0.25 * backoff)
                print(f"{status} server error. Waiting {wait:.2f}s (attempt {attempt}/{max_retries}) ...")
                time.sleep(wait)
                backoff = min(backoff * 2, 60)
                continue
            # non-retryable
            print(f"Non-retryable error {status}: {e}")
            raise
        except Exception as e:
            # network hiccup retry
            wait = backoff + random.uniform(0, 0.25 * backoff)
            print(f"Transient error: {e}. Waiting {wait:.2f}s (attempt {attempt}/{max_retries}) ...")
            time.sleep(wait)
            backoff = min(backoff * 2, 60)
    raise RuntimeError("Max retries exceeded for episodes batch")

# 4) Process in batches of 50, saving each time
batches = [episode_ids_needed[i:i+BATCH_SIZE] for i in range(0, len(episode_ids_needed), BATCH_SIZE)]
print(f"Processing {len(batches)} batches of up to {BATCH_SIZE} episodes each.")

processed_batches = 0
try:
    for ids_batch in tqdm(batches):
        # fetch a batch
        episodes = fetch_batch_with_backoff(sp, ids_batch, market=MARKET)

        # episodes list length == batch length typically; items may be None if invalid
        for ep in episodes:
            if not ep:
                continue
            eid = str(ep.get("id"))
            show = ep.get("show") or {}
            cache[eid] = {
                "show_id": show.get("id"),
                "show_name": show.get("name"),
                "show_publisher": show.get("publisher")
            }

        processed_batches += 1
        if processed_batches % SAVE_EVERY == 0:
            save_checkpoint()

        # small polite delay between batches (optional; keep tiny since you now batch)
        time.sleep(0.2 + random.uniform(0, 0.2))

except KeyboardInterrupt:
    print("⏹️ Interrupted. Saving checkpoint...")
    save_checkpoint()
    raise
except Exception as e:
    print(f"💥 Error: {e}\nSaving checkpoint...")
    save_checkpoint()
    raise
else:
    save_checkpoint()
    print("✅ Done. Filled show fields where available.")


In [6]:
# Saving files
df_matches_filtered.to_csv("data/df_matches_with_showinfo_231025.csv", index=False)
print("Saved to data/df_matches_with_showinfo_231025.csv")


Saved to data/df_matches_with_showinfo_231025.csv


## Clean up to only include shows where guests have overlaps

In [2]:
df_matches_filtered_loaded = pd.read_csv("data/df_matches_with_showinfo_231025.csv")

In [14]:
# Filtering incorrect guest names

# Export list of unique guest names - first dedupe and then export
df_matches_filtered_loaded.guest.drop_duplicates().to_csv("data/unique_guests.csv", index=False)

# Rows to remove if guest name contains: 'Dan Corder and Eugene', 'Anele and Sizwe', 'Microsoft', 'It', 'Idris Elba Walks','Bryan Johnson Wants','Mark Cuban’s','Bill Gates Does'
remove_guests = ['Dan Corder and Eugene', 'Anele and Sizwe', 'Microsoft', 'It', 'Idris Elba Walks','Bryan Johnson Wants','Mark Cuban’s','Bill Gates Does']
df_matches_filtered_loaded = df_matches_filtered_loaded[~df_matches_filtered_loaded['guest'].isin(remove_guests)]
print(len(df_matches_filtered_loaded))


30570


In [15]:
df = df_matches_filtered_loaded.copy()

# 0) hygiene
df = df.dropna(subset=["guest", "show_id"])

# (optional) exclude Trevor’s own podcast so you only see “other” shows
TREVOR_SHOW_ID = "122imavATqSE7eCyXIcqZL"
df = df[df["show_id"] != TREVOR_SHOW_ID]

# 1) dedupe to one row per guest↔show (avoid inflating counts when a guest did multiple episodes)
gs_unique = df.drop_duplicates(subset=["guest", "show_id"])

# 2) find shows with ≥ 2 distinct guests
shows_multi = (
    gs_unique.groupby("show_id")["guest"]
    .nunique()
    .loc[lambda s: s >= 2]
    .index
)

# 3) filter the original df down to only those shows
df_subset = df[df["show_id"].isin(shows_multi)].copy()

# (optional) if you want exactly one row per guest↔show pair in the result:
df_subset_unique = df_subset.drop_duplicates(subset=["guest", "show_id"])

# (optional) quick summary: how many distinct guests per show
summary = (
    df_subset_unique.groupby(["show_id", "show_name"])["guest"]
    .nunique()
    .sort_values(ascending=False)
)
summary.head(20)


show_id                 show_name                                      
7i3PwGmUHVkzQam9ARdcfo  What Now? with Trevor Noah | SiriusXM Podcasts+    45
53dHyhzazFrmPhwuambyuM  The Daily Show: Ears Edition                       25
1VXcH8QHkjRcTCEd88U3ti  TED Talks Daily                                    17
6kAsbP8pxwaU2kPibKTuHE  Armchair Expert with Dax Shepard                   15
3sLR8M7JchYmhWuf8QtlyO  Amanpour                                           15
4ZTHlQzCm7ipnRn1ypnl1Z  The New Yorker Radio Hour                          13
42ntT7XnfdbbYetUQu0vu5  On with Kara Swisher                               12
5RVoEOIACQbBDZnGBJ7Ec2  The Jordan Harbinger Show                          12
3jcdWsWV2KBwsBvBlLCAGx  The Breakfast Club                                 12
2qoc5EIZbcuGUgQhl3QeYd  Talk Easy with Sam Fragoso                         12
3ZK78RyjzfJLWJ41fAaolZ  The Late Show Pod Show with Stephen Colbert        12
4eylg9GZJOVvUhTynt4jjA  Worklife with Adam Grant                      

## Exporting guests and podcasts as nodes and edges

In [16]:
# --- assumptions ---
# df_matches exists with at least: guest, show_id, show_name, episode_id
# optional (used if present): episode_url, show_publisher

# 1) Clean + guard
required = {"guest","show_id","show_name","episode_id"}
missing = required - set(df_subset.columns)
if missing:
    raise RuntimeError(f"df_subset is missing required columns: {missing}")

df = df_subset.dropna(subset=["guest","show_id","show_name"]).copy()
for col in ["guest","show_id","show_name"]:
    df[col] = df[col].astype(str).str.strip()

# 2) Build NODES
guest_nodes = (
    df[["guest"]]
    .drop_duplicates()
    .assign(id=lambda d: "guest:" + d["guest"],
            label=lambda d: d["guest"],
            group="guest",
            url="",
            publisher="")
)[["id","label","group","url","publisher"]]

show_nodes = (
    df[["show_id","show_name"] + (["show_publisher"] if "show_publisher" in df.columns else [])]
    .drop_duplicates()
    .rename(columns={"show_id":"id","show_name":"label",
                     ("show_publisher" if "show_publisher" in df.columns else "show_publisher"): "publisher"})
)
# attach a representative URL per show if available
if "episode_url" in df.columns:
    url_map = df.groupby("show_id")["episode_url"].first().to_dict()
    show_nodes["url"] = show_nodes["id"].map(url_map).fillna("")
else:
    show_nodes["url"] = ""

# ensure publisher column exists
if "publisher" not in show_nodes.columns:
    show_nodes["publisher"] = ""

show_nodes["group"] = "podcast"
show_nodes = show_nodes[["id","label","group","url","publisher"]]

nodes = pd.concat([guest_nodes, show_nodes], ignore_index=True)
nodes = nodes.fillna("")

# 3) Build LINKS (guest↔show edges; weight = episode count)
links = (
    df.groupby(["guest","show_id"], dropna=False)
      .agg(value=("episode_id","count"))
      .reset_index()
)
links["source"] = "guest:" + links["guest"]
links["target"] = links["show_id"]
links = links[["source","target","value"]].astype({"value":"int64"})

# 4) Write to CSV
os.makedirs("data", exist_ok=True)
nodes_path = "data/nodes.csv"
links_path = "data/links.csv"
nodes.to_csv(nodes_path, index=False)
links.to_csv(links_path, index=False)

print(f"✅ Wrote {nodes_path} (nodes={len(nodes)})")
print(f"✅ Wrote {links_path} (links={len(links)})")


✅ Wrote data/nodes.csv (nodes=1455)
✅ Wrote data/links.csv (links=4038)
